MIT License
Copyright (c) 2023 Okyaz Eminaga
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [ ]:
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session
import gc
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
gc.collect()

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))


In [ ]:
from typing import Any, Dict, Iterable, Sequence, Tuple, Optional, Union
from pathlib import Path
from lifelines.utils.concordance import concordance_index
print("Using Tensorflow:", tf.__version__)


In [ ]:
from albumentations import (
    Compose, RandomBrightness, JpegCompression, HueSaturationValue, RandomContrast, HorizontalFlip,
    Rotate, RandomSizedCrop, CenterCrop
)
transforms = Compose([
    Rotate(limit=40),
    RandomBrightness(limit=0.1),
    JpegCompression(quality_lower=85, quality_upper=100, p=0.5),
    HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30,
                       val_shift_limit=20, p=0.5),
    RandomContrast(limit=0.2, p=0.5),
    HorizontalFlip()
])
# For simplicity, we set the center crop to 4096x4096 and the random crop to 512x512.
no_change_transform = Compose([CenterCrop(4096//2, 4096//2, always_apply=True),
                              RandomSizedCrop([512, 586], 512, 512, p=1.0, always_apply=True)])


In [4]:
from PIL import Image


class InputFunction(object):
    """Callable input function that computes the risk set for each batch.

    Parameters
    ----------
    images : np.ndarray, shape=(n_samples, height, width)
        Image data.
    time : np.ndarray, shape=(n_samples,)
        Observed time.
    event : np.ndarray, shape=(n_samples,)
        Event indicator.
    batch_size : int, optional, default=64
        Number of samples per batch.
    drop_last : int, optional, default=False
        Whether to drop the last incomplete batch.
    shuffle : bool, optional, default=False
        Whether to shuffle data.
    seed : int, optional, default=89
        Random number seed.
    """

    def __init__(self,
                 x: np.ndarray,
                 time: np.ndarray,
                 event: np.ndarray,
                 augmentation: bool = False,
                 input_size: (int, int) = (512, 512),
                 channel_number: int = 3,
                 batch_size: int = 32,
                 drop_last: bool = False,
                 shuffle: bool = False,
                 k: int = 1,
                 read_file: bool = False,
                 repeat: int = 1,
                 resize_img: bool = False,
                 seed: int = 89) -> None:
        self.x = x
        self.time = time
        self.input_size = input_size
        self.augmentation = augmentation
        self.event = event
        self.batch_size = batch_size
        self.drop_last = drop_last
        self.shuffle = shuffle
        self.seed = seed
        self.repeat = repeat
        self.k = k
        self.resize_img = resize_img
        self.read_file = read_file
        self.channel_number = channel_number

    def size(self) -> int:
        """Total number of samples."""
        return len(self.x)

    def steps_per_epoch(self) -> int:
        """Number of batches for one epoch."""
        return int(np.floor(len(self.x) / self.batch_size))

    def _get_data_batch(self, index: np.ndarray) -> Tuple[np.ndarray, Dict[str, np.ndarray]]:
        """Compute risk set for samples in batch."""
        time = self.time[index].copy()
        event = self.event[index].copy()
        x = self.x[index].copy()
        if self.read_file:
            images = []
            for fl in x:
                img = Image.open(fl)
                img = img.resize((5120//self.k, 5120//self.k))
                img = np.array(img)
                # img = cv2.resize(img, (5120//self.k, 5120//self.k))
                data = {"image": img}
                if self.augmentation:
                    aug_data = transforms(**data)
                else:
                    aug_data = no_change_transform(**data)
                aug_img = aug_data["image"]
                images.append(aug_img)
            x = np.array(images)
        else:
            if self.resize_img:
                x_tmp = []
                for j in range(x.shape[0]):
                    x_tmp.append(
                        resize(x[j], self.input_size, preserve_range=True).astype(np.uint8))
                x = np.array(x_tmp)
            if self.augmentation:
                for i in range(x.shape[0]):
                    data = {"image": x[i]}
                    aug_data = transforms(**data)
                    x[i] = aug_data["image"]

        return x, event.astype(np.int32)

    def _iter_data(self) -> Iterable[Tuple[np.ndarray, Dict[str, np.ndarray]]]:
        """Generator that yields one batch at a time."""
        index = np.arange(self.size())
        rnd = np.random.RandomState(self.seed)

        if self.shuffle:
            rnd.shuffle(index)
        for b in range(self.steps_per_epoch()):
            start = b * self.batch_size
            idx = index[start:(start + self.batch_size)]
            yield self._get_data_batch(idx)

        if not self.drop_last:
            start = self.steps_per_epoch() * self.batch_size
            idx = index[start:]
            yield self._get_data_batch(idx)

    def _get_shapes(self) -> Tuple[tf.TensorShape, Dict[str, tf.TensorShape]]:
        """Return shapes of data returned by `self._iter_data`."""
        batch_size = self.batch_size if self.drop_last else None
        h, w = self.input_size
        c = self.channel_number
        images = tf.TensorShape([batch_size, h, w, c])
        return images, tf.TensorShape((batch_size,))

    def _get_dtypes(self) -> Tuple[tf.DType, Dict[str, tf.DType]]:
        """Return dtypes of data returned by `self._iter_data`."""
        return tf.float32, tf.int32

    def _make_dataset(self) -> tf.data.Dataset:
        """Create dataset from generator."""
        options = tf.data.Options()
        options.experimental_optimization.noop_elimination = True
        # options.experimental_optimization.map_vectorization.enabled = True
        # options.experimental_optimization.autotune = True
        # options.experimental_optimization.apply_default_optimizations=True
        options.experimental_optimization.map_parallelization = True
        ds = tf.data.Dataset.from_generator(
            self._iter_data,
            self._get_dtypes(),
            self._get_shapes()
        )
        ds = ds.with_options(options)
        if self.repeat > 1:
            return ds.repeat(self.repeat)
        else:
            return ds

    def __call__(self) -> tf.data.Dataset:
        return self._make_dataset()


In [7]:
import numpy as np

time_train = np.load("./time_train_shuffled_20x.npy")
event_train = np.load("./event_train_shuffled_20x.npy")
image_train = np.load("./image_train_shuffled_20x.npy", mmap_mode="r")

time_valid = np.load("./time_valid_shuffled_20x.npy")
event_valid = np.load("./event_valid_shuffled_20x.npy")
image_valid = np.load("./image_valid_shuffled_20x.npy", mmap_mode="r")


In [12]:
train_fn = InputFunction(image_train, time_train, event_train,
                         drop_last=True,
                         augmentation=True,
                         repeat=50,
                         shuffle=True,
                         resize_img=False,
                         input_size=(512, 512),
                         batch_size=16)
eval_fn = InputFunction(image_valid, time_valid, event=event_valid, resize_img=False,
                        input_size=(512, 512))


In [17]:
from plexusnet.architecture import PlexusNet, LoadModel
import tensorflow_addons as tfa
from sklearn.metrics import roc_auc_score
from tensorflow.keras import optimizers


In [21]:
tf.keras.backend.clear_session()
model = PlexusNet(depth=5, length=2, junction=3, n_class=1, final_activation="sigmoid", initial_filter=6, filter_num_for_first_convlayer=4,
                  input_shape=(512, 512), ApplyLayerNormalization=True, run_all_BN=False, type_of_block="soft_att", GlobalPooling="avg").model
model.summary()


In [22]:
lr_sh = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=1e-6,
                                            maximal_learning_rate=1e-3,
                                            step_size=train_fn.steps_per_epoch()*4,
                                            scale_fn=lambda x: 1.,
                                            scale_mode="cycle",
                                            name="MyCyclicScheduler")

model.compile(optimizer=optimizers.Adam(lr_sh), loss=tf.keras.losses.binary_crossentropy,
              metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()])


In [26]:
model_check = tf.keras.callbacks.ModelCheckpoint(
    "./PlexusNET_BCR_20x_BEST_APPROACH_HUE/weight_{epoch:02d}",
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch"
)


In [ ]:
hist = model.fit(train_fn(), steps_per_epoch=train_fn.steps_per_epoch(), epochs=200,
                 validation_steps=eval_fn.steps_per_epoch(), validation_data=eval_fn(),
                 callbacks=[tf.keras.callbacks.CSVLogger('./log_PlexusNET_BCR_20x_BEST_APPROACH_HUE.txt'),
                            model_check])


In [ ]:
import PIL
from tqdm import tqdm
from collections import defaultdict
valid_set = pd.read_csv("valid_set.csv")
test_set = pd.read_csv("./test_set.csv")
valid_set = valid_set[valid_set.Filename.str.contains("/B/") == False]
test_set = test_set[test_set.Filename.str.contains("/B/") == False]
valid_set["X1st.BCR.Type"].value_counts()
valid_set["BCR_status"] = 1-valid_set["X1st.BCR.Type"].str.contains("-")
print(valid_set["X1st.BCR.Type"].value_counts())
print(valid_set["BCR_status"].value_counts())


def GetResult(results):
    case_lst = defaultdict(list)
    y_true_case = defaultdict(list)
    Gls_case = defaultdict(list)
    time_case = defaultdict(list)
    for i, fl in enumerate(results):
        case_id = list(results.keys())[i].split("-")[2]
        case_lst[case_id].extend(results[fl])
        time_case[case_id].append(
            valid_set["Interval.RP.to.BCR.or.last.contact.death"].iloc[i])
        y_true_case[case_id].append(valid_set.BCR_status.iloc[i])
        Gls_case[case_id].append(list(results.keys())[i].split("/")[2])
    y_true_lst = []
    y_pred_lst = []
    y_time_lst = []
    for key in y_true_case:
        y_true_lst.append(y_true_case[key][0])
        y_ = np.histogram(case_lst[key])
        b = np.where(y_[0] >= 2)
        _m = np.max(y_[1][b])
        y_pred_lst.append(_m)
        y_time_lst.append(time_case[key][0])
    print(roc_auc_score(y_true_lst, y_pred_lst), concordance_index(
        y_time_lst, 1-np.array(y_pred_lst), y_true_lst))
    return {'roc': roc_auc_score(y_true_lst, y_pred_lst),
            'cindex': concordance_index(y_time_lst, 1-np.array(y_pred_lst), y_true_lst)}


def RunAnalyses(model_best):
    results = defaultdict(list)
    heatmaps = defaultdict(list)
    for fl in tqdm(valid_set.Filename):
        img = np.array(PIL.Image.open(fl).resize((5120//2, 5120//2)))
        img = img[128:-128, 128:-128]
        heatmap = np.zeros((3, 3), dtype=np.float)
        patch = []
        for j in range(0, img.shape[0]-256, 256):
            for i in range(0, img.shape[1]-256, 256):
                patch.append(img[j:j+512, i:i+512])
        pr = model_best.predict(np.array(patch), verbose=0)
        k = 0
        results[fl] = pr
    return heatmaps, results


# Run Analyze and
heatmaps_model = {}
results_model = {}
cindex_model = {}
roc_model = {}
for epoch in range(1, 201):
    print(epoch)
    model_best = LoadModel(
        f"PlexusNET_BCR_20x_BEST_APPROACH_HUE/weight_{epoch:02d}")
    heatmaps, results = RunAnalyses(model_best)
    v = GetResult(results)
    heatmaps_model[epoch] = heatmaps
    results_model[epoch] = results
    cindex_model[epoch] = v["cindex"]
    roc_model[epoch] = v["roc"]

# SELEC BEST EPOCH
ind_x = np.argmax(list(cindex_model.values()))
print(ind_x+1)
print(list(cindex_model.values())[ind_x])
print(list(roc_model.values())[ind_x])


In [1]:
from plexusnet.architecture import PlexusNet, LoadModel
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from plexusnet.architecture import *
import keras
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session
import numpy as np
import gc
import os
gc.collect()


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))

tf.keras.backend.clear_session()


In [2]:
modelx = "PlexusNET_BCR_20x_BEST_APPROACH_HUE"
weight_file = "weight_27"


In [3]:
test_set = pd.read_csv("./test_set_OnlyTumor.csv")


In [4]:
development_set = pd.read_csv("./development_set_OnlyTumor.csv")


In [6]:
data_path = "./dataset/"


def RunAnalyses(model_best, dataset):
    results = defaultdict(list)
    heatmaps = defaultdict(list)
    for fl_ in tqdm(dataset.Filename):

        fl = data_path+fl_[1:]
        img = np.array(PIL.Image.open(fl).resize((5120//2, 5120//2)))
        img = img[128:-128, 128:-128]
        heatmap = np.zeros((3, 3), dtype=np.float)
        patch = []
        for j in range(0, img.shape[0]-256, 256):
            for i in range(0, img.shape[1]-256, 256):
                img_C = np.array(PIL.Image.fromarray(
                    img[j:j+512, i:i+512]), dtype=np.uint8)
                patch.append(img_C)
        pr = model_best.predict(np.array(patch), verbose=0)
        k = 0
        '''
        for j in range(0,3):
            for i in range(0, 3):
                heatmap[j,i]=pr[k]
                k+=1
        '''
        heatmaps[fl] = heatmap
        results[fl] = pr
    return heatmaps, results


In [8]:
tf.keras.backend.clear_session
model_best = LoadModel(f"./{modelx}/{weight_file}")


In [ ]:
_, results_developmentset = RunAnalyses(model_best, development_set)


In [ ]:
_, results_testset = RunAnalyses(model_best, test_set)


In [11]:
columns_developmentset = defaultdict(list)
for j in range(64):
    for k in results_developmentset:
        columns_developmentset[j].append(
            results_developmentset[k][j].flatten()[0])


In [12]:
columns_test = defaultdict(list)
for j in range(64):
    for k in results_testset:
        columns_test[j].append(results_testset[k][j].flatten()[0])


In [13]:
for col in columns_test:
    test_set[col] = columns_test[col]


In [14]:
test_set.to_csv(f"{modelx}_test_set.csv")


In [15]:
for col in columns_developmentset:
    development_set[col] = columns_developmentset[col]


In [16]:
development_set.to_csv(f"{modelx}_development_set.csv")
